In [13]:
import json
import pprint

In [4]:
with open('data/synthetic_data.json', 'r') as f:
    synthetic_data = json.load(f)

In [14]:
pprint.pprint(synthetic_data)

{'agency': {'agencyId': 3084503,
            'contactId': 0,
            'email': 'cruiseres@trailfinders.com',
            'invoiceType': 'To Agency',
            'name': 'Trailfinders',
            'phone': '02073683840'},
 'bookingArea': 'UK',
 'bookingCurrency': 'GBP',
 'bookingId': 3000350,
 'bookingSFID': '801So00000F4jZ3IAJ',
 'bookingSource': 'PGCLIENT',
 'brochure': 'UKEXI26',
 'canAutoCancel': False,
 'cancellationDateTime': None,
 'cancellationReason': None,
 'company': 'HRX',
 'createdBy': 'Angela.Kurz@travelhx.com',
 'createdDateTime': '2024-11-07T19:00:14.91',
 'departureDateTime': '2026-07-23T14:50:00',
 'depositDueDate': '2024-11-14T19:00:14.91',
 'endDateTime': '2026-07-31T08:30:00',
 'finalDueDate': '2026-04-22T00:00:00',
 'financialOverview': {'depositAmount': 2778.825,
                       'grossBalance': 18525.5,
                       'netBalance': 13412.46,
                       'netPrice': 13412.46,
                       'totalCommission': 4260.87,
         

### Tour Date Info

In [42]:
tour_start_date = synthetic_data['startDateTime']
tour_start_year = tour_start_date.split('-')[0]
tour_start_month = {
    '01': 'January',
    '02': 'February', 
    '03': 'March',
    '04': 'April',
    '05': 'May',
    '06': 'June',
    '07': 'July',
    '08': 'August',
    '09': 'September',
    '10': 'October',
    '11': 'November',
    '12': 'December'
}[tour_start_date.split('-')[1]]
tour_start_day = tour_start_date.split('-')[2]

In [43]:
tour_start_month

'March'

### Getting Passenger Info

In [17]:
passenger_details = synthetic_data['passengerGroups']

In [23]:
passenger_info = passenger_details[0]["passengers"]

In [24]:
number_of_passengers = len(passenger_info)

In [25]:
number_of_passengers

2

In [30]:
def get_passenger_primary_info(passenger_info):
    return {
        "name": passenger_info["name"],
        "email": passenger_info.get("email", ""),
        "phone": passenger_info.get("phone", ""),
        "address": passenger_info.get("address", "")
    }

In [31]:
passenger_primary_info = []
for passenger in passenger_info:
    passenger_primary_info.append(get_passenger_primary_info(passenger))

In [32]:
passenger_primary_info

[{'name': 'FREDI KRUGER',
  'email': 'test@test.com',
  'phone': '',
  'address': ''},
 {'name': 'TEST JEAN PIERRE LAFFONT',
  'email': 'test.fr@icloud.com',
  'phone': '',
  'address': ''}]

In [33]:
### Get Cabin Info

In [39]:
def get_cabin_info(cabin):
    return {
        "category": cabin.get("category"),
        "cabin_number": cabin.get("cabinNumber"),
        "seaware_stage_type": cabin.get("passengerDetails")[0].get("seawareStageType")
    }


In [40]:
cabin_info = []
for cabin in passenger_details[0]["sailings"][0]["cabins"]:
    cabin_info.append(get_cabin_info(cabin))
cabin_info

[{'category': 'N2',
  'cabin_number': '336',
  'seaware_stage_type': 'Outside Cabin'},
 {'category': 'N2',
  'cabin_number': '336',
  'seaware_stage_type': 'Outside Cabin'}]

In [41]:
### Construct Prompt

In [50]:
prompt = f"""
- Navigate to https://sandbox.reservations.travelhx.com/touch
- Login using user name VR_Patrick and password SEAWARE_PASSWORD
- Click New Reservation at bottom of screen
- Click on element by index with id 15
- In the calendar under "Select the tour you prefer from the list below", click on the second year dropdown where it says 2025 and click on {tour_start_year} using the select_dropdown_option function
- In the dropdown for month, using the select_dropdown_option function, click on the month where it says {tour_start_month}
- Click on the tile on the day {tour_start_day} of the month. It should be highlighted in blue once selected
- Find the select checkbox and pick it. It should be in the bottom right of the screen (below the cost)
- Once selected a green tick should appear
- Click continue
- Find the row for {cabin_info[0]["seaware_stage_type"]}. It has code {cabin_info[0]["category"]}. It should have this exact name. The cabins have their own scroll section. Do a ctrl + f find for the term {cabin_info[0]["seaware_stage_type"]}
- Click on the plus button on this row so that a 1 should appear
- Click continue
- Click the "Change" button next to the number 
- Search for the cabin {cabin_info[0]["cabin_number"]} in the Stateroom box by typing it in and pressing enter
- Press Click Select to select the cabin {cabin_info[0]["cabin_number"]}
"""

In [49]:
prompt

'\n- Navigate to https://sandbox.reservations.travelhx.com/touch\n- Login using user name VR_Patrick and password SEAWARE_PASSWORD\n- Click New Reservation at bottom of screen\n- Click on Find Tours, so it is minimised\n- Scroll down if it isn\'t\n- In the calendar under "Select the tour you prefer from the list below", click on the second year dropdown where it says 2025 and click on 2028 using the select_dropdown_option function\n- In the dropdown for month, using the select_dropdown_option function, click on the month where it says March\n- Click on the tile on the day 18T00:00:00 of the month. It should be highlighted in blue once selected\n- Find the select checkbox and pick it. It should be in the bottom right of the screen (below the cost)\n- Once selected a green tick should appear\n- Click continue\n- Find the row for N2. It has code 336. It should have this exact name. The cabins have their own scroll section. Do a ctrl + f find for the term N2\n- Click on the plus button on 